In [1]:
import pandas as pd
import datetime as dt
import re

data = pd.read_excel(r'C:\Users\Seal\PycharmProjects\Digital office\Digital office.xlsx')
data.head()

,counterid,event_id,eventtype,reg_event_ts,referrer,title,uid,url
0,4,7916,1,2021-03-19 09:05:36,https://kb.gazprom-neft.local/pages/viewpage.a...,Общая информация Новая Голландия - Digital off...,3fe9a0c55cef9f75e77eac40a631bee8b99ef43190dc49...,https://kb.gazprom-neft.local/pages/viewpage.a...
1,4,9233,1,2021-03-19 17:14:46,NaN,Digital office / Офис - Digital office - Gazpr...,3fe9a0c55cef9f75e77eac40a631bee8b99ef43190dc49...,https://kb.gazprom-neft.local/display/DO
2,4,10269,1,2021-03-22 02:39:16,https://kb.gazprom-neft.local/spacedirectory/v...,Digital office / Офис - Digital office - Gazpr...,a45cb38e43026eaa0edf55a145e9e269fe6b02d2ca158e...,https://kb.gazprom-neft.local/pages/viewpage.a...
3,4,10382,1,2021-03-21 21:37:31,https://kb.gazprom-neft.local/spacedirectory/v...,Digital office / Офис - Digital office - Gazpr...,3fe9a0c55cef9f75e77eac40a631bee8b99ef43190dc49...,https://kb.gazprom-neft.local/pages/viewpage.a...
4,4,11665,1,2021-03-22 10:59:06,https://kb.gazprom-neft.local/,Digital office / Офис - Digital office - Gazpr...,7b7f2ddc5c21ec118a35c2e1eeb8505246c36d5a2e2873...,https://kb.gazprom-neft.local/pages/viewpage.a...


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   counterid     32135 non-null  int64         
 1   event_id      32135 non-null  int64         
 2   eventtype     32135 non-null  int64         
 3   reg_event_ts  32135 non-null  datetime64[ns]
 4   referrer      25325 non-null  object        
 5   title         32135 non-null  object        
 6   uid           32135 non-null  object        
 7   url           32135 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 2.0+ MB


In [3]:
data['event_date'] = data['reg_event_ts'].dt.date
#data['mnth_yr'] = data['event_date'].apply(lambda x: x.strftime('%B-%Y'))
data['month'] = data['reg_event_ts'].dt.month
data['year'] = data['reg_event_ts'].dt.year
data = data.query('reg_event_ts >= "2022-01-01" and reg_event_ts < "2023-02-01"')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22360 entries, 9490 to 31849
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   counterid     22360 non-null  int64         
 1   event_id      22360 non-null  int64         
 2   eventtype     22360 non-null  int64         
 3   reg_event_ts  22360 non-null  datetime64[ns]
 4   referrer      17060 non-null  object        
 5   title         22360 non-null  object        
 6   uid           22360 non-null  object        
 7   url           22360 non-null  object        
 8   event_date    22360 non-null  object        
 9   month         22360 non-null  int64         
 10  year          22360 non-null  int64         
dtypes: datetime64[ns](1), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
data['referrer'] = data['referrer'].fillna(value = "Прямой")
data['referrer_new'] = [x.split("//")[1] if len(x.split("//")) == 2 else x for x in data['referrer']]
data['transition'] = [x.split(".")[0] for x in data['referrer_new']]
data['referrer_new'] = [x.split("/")[0] for x in data['referrer_new']]

In [5]:
data['referrer_new'].value_counts()

kb.gazprom-neft.local         16743
Прямой                         5300
search.gazprom-neft.local       195
task.gazprom-neft.local          87
lenta.gazprom-neft.ru            13
sm.gazprom-neft.local            11
owebapp.gazprom-neft.local       10
task-t.gazprom-neft.local         1
Name: referrer_new, dtype: int64

In [6]:
data['transition'].value_counts()

kb         16743
Прямой      5300
search       195
task          87
lenta         13
sm            11
owebapp       10
task-t         1
Name: transition, dtype: int64

In [7]:
def transition(cell):
    if 'kb' == cell:
        return 'Confluence'
    if 'search' == cell:
        return 'Корпоративный поиск'
    if 'task' in cell:
        return 'Jira'
    if 'lenta' == cell:
        return 'Лента'
    if 'sm' == cell:
        return 'Портал IaaS'
    if 'Прямой' == cell:
        return 'Прямой'
    return 'Прочее'

In [8]:
data['transition'] = data['transition'].apply(transition)
data['transition'].value_counts(dropna = False)

Confluence             16743
Прямой                  5300
Корпоративный поиск      195
Jira                      88
Лента                     13
Портал IaaS               11
Прочее                    10
Name: transition, dtype: int64

In [9]:
data['title'].value_counts()

Digital office / Офис - Digital office - Gazprom Neft Confluence                                 2908
Памятка руководителю / наставнику - Digital office - Gazprom Neft Confluence                      836
📆 Запись на спортивные занятия Цифергауз - Digital office - Gazprom Neft Confluence               637
🏢 Рассадка "Невская Ратуша" - Digital office - Gazprom Neft Confluence                            621
Концепция распределения рабочих мест для ДЦТ и ДИР - Digital office - Gazprom Neft Confluence     488
                                                                                                 ... 
13.07.2021 Результаты встречи - Digital office - Gazprom Neft Confluence                            1
Изменить - Спортивный зал - Digital office - Gazprom Neft Confluence                                1
Ретроспективы - Digital office - Gazprom Neft Confluence                                            1
Изменить - 12345678 - Digital office - Gazprom Neft Confluence                    

In [10]:
len(data['title'].value_counts())

342

In [11]:
data['title_new'] = [x.split(" - D")[0] for x in data['title']]

In [12]:
data['title_new'].value_counts().head(20)

Digital office / Офис                                              2908
Памятка руководителю / наставнику                                   836
📆 Запись на спортивные занятия Цифергауз                            637
🏢 Рассадка "Невская Ратуша"                                         621
Концепция распределения рабочих мест для ДЦТ и ДИР                  488
Памятка новому сотруднику                                           437
🏢 Офис "Цифергауз" на о. Новая Голландия                            416
Рассадка "Невская Ратуша"                                           387
Рассадка "Цифергауз"                                                370
Сценарии пропускного режима                                         358
Офис "Цифергауз" на о. Новая Голландия                              347
Спортивные занятия Цифергауз                                        341
📋 Распределение рабочих мест для ДЦТ, ДИР, ДЦИ                      328
🏢 Рассадка "Цифергауз"                                          

In [13]:
data['title_new'] = [cell if cell[0].isalpha()==True or cell[0].isdigit()==True else cell[2:] for cell in data['title_new']]
data['title_new'] = [cell[2:] if cell[0] == 'ℹ' else cell for cell in data['title_new']]

In [14]:
data['title_new'].value_counts().head(30)

Digital office / Офис                                            2908
Рассадка "Невская Ратуша"                                        1008
Памятка руководителю / наставнику                                 836
Запись на спортивные занятия Цифергауз                            817
Офис "Цифергауз" на о. Новая Голландия                            763
Рассадка "Цифергауз"                                              693
Сценарии пропускного режима                                       514
Среда здоровья                                                    492
Концепция распределения рабочих мест для ДЦТ и ДИР                488
Общая информация Невская Ратуша                                   454
Памятка новому сотруднику                                         437
Спортивные занятия Цифергауз                                      341
Незакрепленные рабочие места                                      336
Распределение рабочих мест для ДЦТ, ДИР, ДЦИ                      328
Офис "Дом Инноваций"

In [15]:
data['url_new'] = data['url'].apply(lambda x: re.split("(\d{8,9})", x)[0] + re.split("(\d{8,9})", x)[1] 
                                    if re.search("(\d{8,9})", x) == True else x)
data['url_new'] = [x.split("&")[0] for x in data['url_new']]

In [16]:
data['url_new'].value_counts().head(20)

https://kb.gazprom-neft.local/pages/viewpage.action?pageId=8199304      2038
https://kb.gazprom-neft.local/pages/viewpage.action?pageId=85173667     1121
https://kb.gazprom-neft.local/pages/viewpage.action?pageId=58494363      999
https://kb.gazprom-neft.local/display/DO                                 845
https://kb.gazprom-neft.local/pages/viewpage.action?pageId=30905195      835
https://kb.gazprom-neft.local/pages/viewpage.action?pageId=23973894      826
https://kb.gazprom-neft.local/pages/viewpage.action?pageId=33201138      762
https://kb.gazprom-neft.local/pages/viewpage.action?pageId=47421953      684
https://kb.gazprom-neft.local/pages/viewpage.action?pageId=38963286      593
https://kb.gazprom-neft.local/pages/viewpage.action?pageId=33201136      494
https://kb.gazprom-neft.local/pages/viewpage.action?pageId=45812092      487
https://kb.gazprom-neft.local/pages/viewpage.action?pageId=23969935      454
https://kb.gazprom-neft.local/pages/viewpage.action?pageId=23968289      432

In [17]:
url_dict = {'https://kb.gazprom-neft.local/pages/viewpage.action?pageId=33201136':'Офис "Дом Инноваций" в Невской Ратуше',
            'https://kb.gazprom-neft.local/pages/viewpage.action?pageId=23969935':'Общая информация Невская Ратуша',
            'https://kb.gazprom-neft.local/pages/viewpage.action?pageId=90529891':'Сервисная служба Невской Ратуши',
            'https://kb.gazprom-neft.local/pages/viewpage.action?pageId=23969955':'Офисные сервисы Невская Ратуша',
            'https://kb.gazprom-neft.local/pages/viewpage.action?pageId=23970285':'Командная работа и встречи в Невской Ратуше',
            'https://kb.gazprom-neft.local/pages/viewpage.action?pageId=23961147':'Общественные пространства и отдых в Невской Ратуше',
            'https://kb.gazprom-neft.local/pages/viewpage.action?pageId=23970326':'Массовые и официальные мероприятия',
            'https://kb.gazprom-neft.local/pages/viewpage.action?pageId=23969960':'Где поесть в Невской Ратуше',
            'https://kb.gazprom-neft.local/pages/viewpage.action?pageId=35994650':'Экология в офисе',
            'https://kb.gazprom-neft.local/pages/viewpage.action?pageId=33201138':'Офис "Цифергауз" на о. Новая Голландия',
            'https://kb.gazprom-neft.local/pages/viewpage.action?pageId=23968257':'Общая информация Цифергауз',
            'https://kb.gazprom-neft.local/pages/viewpage.action?pageId=90532265':'Сервисная служба Цифергауза',
            'https://kb.gazprom-neft.local/pages/viewpage.action?pageId=23969113':'Офисные сервисы Цифергауз',
            'https://kb.gazprom-neft.local/pages/viewpage.action?pageId=38963286':'Сценарии пропускного режима',
            'https://kb.gazprom-neft.local/pages/viewpage.action?pageId=124430065':'Командная работа и встречи в Цифергауз (переговорные комнаты)',
            'https://kb.gazprom-neft.local/pages/viewpage.action?pageId=23968366':'Общественные пространства и отдых в Цифергаузе',
            'https://kb.gazprom-neft.local/pages/viewpage.action?pageId=23969133':'Массовые и официальные мероприятия в Цифергауз',
            'https://kb.gazprom-neft.local/pages/viewpage.action?pageId=23968282':'Где поесть в Новой Голландии',
            'https://kb.gazprom-neft.local/pages/viewpage.action?pageId=36005102':'Экология в офисе Цифергауз',
            'https://kb.gazprom-neft.local/pages/viewpage.action?pageId=90533579':'Офис ИТ-кластера в Сириусе',
            'https://kb.gazprom-neft.local/pages/viewpage.action?pageId=124431938':'Общая информация об офисе в Сириусе',
            'https://kb.gazprom-neft.local/pages/viewpage.action?pageId=108335771':'Офисные сервисы',
            'https://kb.gazprom-neft.local/pages/viewpage.action?pageId=108335865':'Общественные пространства в "Дельта Сириус"',
            'https://kb.gazprom-neft.local/pages/viewpage.action?pageId=104534333':'Как поехать в Сочи? Порядок номинирования команд и бронирования офиса',
            'https://kb.gazprom-neft.local/pages/viewpage.action?pageId=104557882':'Как оплатить мероприятие/поездку?',
            'https://kb.gazprom-neft.local/pages/viewpage.action?pageId=121904884':'Варианты командообразующих мероприятий',
            'https://kb.gazprom-neft.local/pages/viewpage.action?pageId=47421953':'Рассадка "Цифергауз"',
            'https://kb.gazprom-neft.local/pages/viewpage.action?pageId=58494363':'Рассадка "Невская Ратуша"'
}

In [18]:
data_fav = data.query('url_new in @url_dict.keys()')
data_not = data.query('url_new not in @url_dict.keys()')

In [19]:
len(data_not)

14611

In [20]:
data_fav['title_new'] = data_fav['url_new'].map(url_dict)

<ipython-input-20-aff3211fb08f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fav['title_new'] = data_fav['url_new'].map(url_dict)


In [21]:
data_fav['title_new'].value_counts()

Рассадка "Невская Ратуша"                                                999
Офис "Цифергауз" на о. Новая Голландия                                   762
Рассадка "Цифергауз"                                                     684
Сценарии пропускного режима                                              593
Офис "Дом Инноваций" в Невской Ратуше                                    494
Общая информация Невская Ратуша                                          454
Офис ИТ-кластера в Сириусе                                               405
Общая информация Цифергауз                                               313
Офисные сервисы Цифергауз                                                295
Офисные сервисы Невская Ратуша                                           274
Общественные пространства и отдых в Цифергаузе                           272
Массовые и официальные мероприятия в Цифергауз                           244
Массовые и официальные мероприятия                                       240

In [22]:
data = data_fav.append(data_not, ignore_index = True).sort_values(by = 'reg_event_ts').reset_index()

In [23]:
data['title_new'].value_counts().head(30)

Digital office / Офис                                            2908
Рассадка "Невская Ратуша"                                        1008
Памятка руководителю / наставнику                                 836
Запись на спортивные занятия Цифергауз                            817
Офис "Цифергауз" на о. Новая Голландия                            763
Рассадка "Цифергауз"                                              693
Сценарии пропускного режима                                       671
Офис "Дом Инноваций" в Невской Ратуше                             494
Среда здоровья                                                    492
Концепция распределения рабочих мест для ДЦТ и ДИР                488
Общая информация Невская Ратуша                                   454
Памятка новому сотруднику                                         437
Офис ИТ-кластера в Сириусе                                        405
Спортивные занятия Цифергауз                                      341
Незакрепленные рабоч

In [24]:
def title(cell):
    if 'Офис \"Дом Инноваций\" в' in cell:
        return 'Офис "Дом Инноваций" в Невской Ратуше'
    if 'Офис ИТ-кластера в Сириусе' in cell or 'Временный офис ИТ-кластера в Сириусе' in cell:
        return 'Офис ИТ-кластера в Сириусе'
    if 'Командная работа и встречи в Цифергауз' in cell or 'Переговорные комнаты' == cell:
        return 'Командная работа и встречи в Цифергауз (переговорные комнаты)'
    if 'Сценарии пропускного режима' in cell:
        return 'Сценарии пропускного режима'
    if 'Как оплатить мероприятие' in cell:
        return 'Как оплатить мероприятие/поездку?'
    if 'Как поехать в Сочи' in cell:
        return 'Как поехать в Сочи? Порядок номинирования команд и бронирования офиса'
    if 'Общая информация Новая Голландия' in cell:
        return 'Общая информация Цифергауз'
    if 'Общественные пространства и отдых в Новой Голландии' in cell:
        return 'Общественные пространства и отдых в Цифергаузе'
    if 'Офис "Дом Инноваций" в БЦ Невская Ратуша' in cell:
        return 'Офис "Дом Инноваций" в Невской Ратуше'
    if 'Офисные сервисы Новая Голландия' == cell or 'Офисные сервисы "Цифергауз"'== cell:
        return 'Офисные сервисы Цифергауз'
    if 'Сервисная служба Новой Голландии' in cell:
        return 'Сервисная служба Цифергауза'
    if 'Сервисная служба' == cell:
        return 'Сервисная служба Невской Ратуши'
    return cell

In [25]:
data['title_new'] = data['title_new'].apply(title)

In [26]:
data['title_new'].value_counts().head(30)

Digital office / Офис                                            2908
Рассадка "Невская Ратуша"                                        1008
Памятка руководителю / наставнику                                 836
Запись на спортивные занятия Цифергауз                            817
Офис "Цифергауз" на о. Новая Голландия                            763
Сценарии пропускного режима                                       695
Рассадка "Цифергауз"                                              693
Командная работа и встречи в Цифергауз (переговорные комнаты)     501
Офис "Дом Инноваций" в Невской Ратуше                             496
Среда здоровья                                                    492
Концепция распределения рабочих мест для ДЦТ и ДИР                488
Общая информация Невская Ратуша                                   454
Памятка новому сотруднику                                         437
Офис ИТ-кластера в Сириусе                                        410
Спортивные занятия Ц

In [27]:
data['fav'] = data['title_new'].isin(url_dict.values())

In [28]:
data.query('fav == True')['title_new'].value_counts()

Рассадка "Невская Ратуша"                                                1008
Офис "Цифергауз" на о. Новая Голландия                                    763
Сценарии пропускного режима                                               695
Рассадка "Цифергауз"                                                      693
Командная работа и встречи в Цифергауз (переговорные комнаты)             501
Офис "Дом Инноваций" в Невской Ратуше                                     496
Общая информация Невская Ратуша                                           454
Офис ИТ-кластера в Сириусе                                                410
Общая информация Цифергауз                                                313
Офисные сервисы Цифергауз                                                 295
Офисные сервисы Невская Ратуша                                            274
Общественные пространства и отдых в Цифергаузе                            273
Массовые и официальные мероприятия в Цифергауз                  

In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22360 entries, 0 to 22359
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   index         22360 non-null  int64         
 1   counterid     22360 non-null  int64         
 2   event_id      22360 non-null  int64         
 3   eventtype     22360 non-null  int64         
 4   reg_event_ts  22360 non-null  datetime64[ns]
 5   referrer      22360 non-null  object        
 6   title         22360 non-null  object        
 7   uid           22360 non-null  object        
 8   url           22360 non-null  object        
 9   event_date    22360 non-null  object        
 10  month         22360 non-null  int64         
 11  year          22360 non-null  int64         
 12  referrer_new  22360 non-null  object        
 13  transition    22360 non-null  object        
 14  title_new     22360 non-null  object        
 15  url_new       22360 non-null  object

In [30]:
data_fav = data.query('fav == True and reg_event_ts < "2023-02-01"')[['event_id', 
                                    'event_date', 'month', 'year', 'referrer_new', 'transition', 'title_new', 'uid', 'url_new']]
data_all = data[['event_id', 'event_date', 'month', 'year', 'referrer_new', 'transition', 'title_new', 'uid', 'url_new', 'fav']]

In [31]:
data_fav.head()

,event_id,event_date,month,year,referrer_new,transition,title_new,uid,url_new
3,4829519,2022-01-10,1,2022,kb.gazprom-neft.local,Confluence,Где поесть в Невской Ратуше,987b6c531103a69837ab47091f2cdf649026d540559a9d...,https://kb.gazprom-neft.local/pages/viewpage.a...
10,4829792,2022-01-10,1,2022,kb.gazprom-neft.local,Confluence,"Офис ""Дом Инноваций"" в Невской Ратуше",987b6c531103a69837ab47091f2cdf649026d540559a9d...,https://kb.gazprom-neft.local/pages/viewpage.a...
11,4829793,2022-01-10,1,2022,kb.gazprom-neft.local,Confluence,Где поесть в Невской Ратуше,987b6c531103a69837ab47091f2cdf649026d540559a9d...,https://kb.gazprom-neft.local/pages/viewpage.a...
12,4829797,2022-01-10,1,2022,kb.gazprom-neft.local,Confluence,"Офис ""Дом Инноваций"" в Невской Ратуше",987b6c531103a69837ab47091f2cdf649026d540559a9d...,https://kb.gazprom-neft.local/pages/viewpage.a...
13,4829798,2022-01-10,1,2022,kb.gazprom-neft.local,Confluence,Офисные сервисы Невская Ратуша,987b6c531103a69837ab47091f2cdf649026d540559a9d...,https://kb.gazprom-neft.local/pages/viewpage.a...


In [32]:
data_fav.pivot_table(index = 'title_new', columns = ['year', 'month'], values = 'event_id', aggfunc = 'count')

year                                                2022                     \
month                                                 1     2      3     4    
title_new                                                                     
Варианты командообразующих мероприятий               NaN   NaN    NaN   NaN   
Где поесть в Невской Ратуше                          6.0   5.0   20.0   7.0   
Где поесть в Новой Голландии                         3.0   5.0   21.0   6.0   
Как оплатить мероприятие/поездку?                    NaN   NaN    NaN   NaN   
Как поехать в Сочи? Порядок номинирования коман...   NaN   NaN    NaN   NaN   
Командная работа и встречи в Невской Ратуше          2.0  11.0   17.0   5.0   
Командная работа и встречи в Цифергауз (перегов...  12.0  21.0   26.0  24.0   
Массовые и официальные мероприятия                   3.0   2.0   17.0   3.0   
Массовые и официальные мероприятия в Цифергауз      10.0   6.0   28.0  12.0   
Общая информация Невская Ратуша                     21.0  39.0   41.0  18.0   
Общая информация Цифергауз                          29.0  17.0   48.0  13.0   
Общая информация об офисе в Сириусе                  NaN   NaN    NaN   NaN   
Общественные пространства в "Дельта Сириус"          NaN   NaN    NaN   NaN   
Общественные пространства и отдых в Невской Ратуше   6.0   7.0   13.0   8.0   
Общественные пространства и отдых в Цифергаузе       5.0  11.0   29.0  12.0   
Офис "Дом Инноваций" в Невской Ратуше               18.0  12.0   43.0  15.0   
Офис "Цифергауз" на о. Новая Голландия              39.0  24.0   77.0  29.0   
Офис ИТ-кластера в Сириусе                           NaN   NaN   11.0  11.0   
Офисные сервисы                                      NaN   NaN    NaN   NaN   
Офисные сервисы Невская Ратуша                      12.0  10.0   28.0   8.0   
Офисные сервисы Цифергауз                           28.0  11.0   48.0  11.0   
Рассадка "Невская Ратуша"                           47.0  35.0  102.0  72.0   
Рассадка "Цифергауз"                                37.0  60.0   98.0  50.0   
Сервисная служба Невской Ратуши                      NaN   NaN   10.0   NaN   
Сервисная служба Цифергауза                          NaN   NaN    4.0   1.0   
Сценарии пропускного режима                         32.0  30.0   41.0  18.0   
Экология в офисе                                     4.0   2.0    9.0   1.0   
Экология в офисе Цифергауз                           1.0   1.0    5.0   3.0   

year                                                                         \
month                                                 5     6      7     8    
title_new                                                                     
Варианты командообразующих мероприятий               NaN   NaN    NaN  19.0   
Где поесть в Невской Ратуше                          6.0  10.0   15.0  18.0   
Где поесть в Новой Голландии                         7.0   8.0   29.0  15.0   
Как оплатить мероприятие/поездку?                    NaN  12.0   26.0   9.0   
Как поехать в Сочи? Порядок номинирования коман...   NaN  30.0   31.0  10.0   
Командная работа и встречи в Невской Ратуше          3.0   4.0   33.0  30.0   
Командная работа и встречи в Цифергауз (перегов...  29.0  23.0   69.0  71.0   
Массовые и официальные мероприятия                   NaN   5.0   30.0  38.0   
Массовые и официальные мероприятия в Цифергауз      17.0   9.0   27.0  21.0   
Общая информация Невская Ратуша                     11.0  21.0   40.0  35.0   
Общая информация Цифергауз                          12.0  14.0   31.0  23.0   
Общая информация об офисе в Сириусе                  NaN   NaN    NaN   NaN   
Общественные пространства в "Дельта Сириус"          NaN   NaN   16.0   4.0   
Общественные пространства и отдых в Невской Ратуше   8.0  11.0   24.0  16.0   
Общественные пространства и отдых в Цифергаузе      11.0  14.0   45.0  30.0   
Офис "Дом Инноваций" в Невской Ратуше               11.0  28.0   46.0  38.0   
Офис "Цифергауз" на о. Новая Голландия           

In [33]:
data_all.to_excel('data_clean.xlsx')
data_fav.to_excel('data_favorite.xlsx')